In [1]:
import logging
import sys

logging.basicConfig(format='%(levelname)s | %(message)s',
                    level=logging.DEBUG,
                    stream=sys.stdout)
log = logging.getLogger(__name__)

In [2]:
import datalink

In [3]:
datalink.test_output()

INFO | logging from datalink


In [4]:
datalink.SQLInterface(db_path='t2.db')

In [5]:
import os.path
os.path.expandvars('~/Downloads')

'~/Downloads'

In [6]:
from pathlib import Path
Path('~/Downloads/').expanduser()

PosixPath('/home/sogilvy/Downloads')